In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.io
import pandas as pd
from numpy import genfromtxt
from sklearn.model_selection import *
from sklearn.svm import SVC
from sklearn.metrics import *
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.manifold import TSNE
import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import *
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import cv2
from torchmeta.modules import *
from collections import OrderedDict
from torch.utils.data import TensorDataset, DataLoader
from torchmeta.utils.data import *
import random
import maml
from maml.metalearners import ModelAgnosticMetaLearning
from torchmeta.utils.data import MetaDataset
import math


In [2]:
%load_ext autoreload
%autoreload 2

## README.txt

This folder contains the preliminary dataset for the coursework for R250: Applications of ML to Psychiatry.

COBRE_demographics.csv includes a list of subject ids, labels, sex and age information.
Labels: -1 for a healthy control subject, 1 for a patient with schizophrenia
Sex: -1 for male, 1 for female
PANSS total: these are symptom scores (from the Positive and Negative Syndrome Scale). Note controls do not have symptom data. Symptoms are also missing for 2 patients.

COBRE_cortical_thickness.csv includes cortical thickness measurements for all subjects, for 308 cortical brain regions.

COBRE_fmri_connectivity.mat is a Matlab file with connectivity data for all subjects. Subjects are listed in the same order as the demographics file above.
Note that not only 293 out of 308 brain regions have fmri data available (due to incomplete fMRI coverage of the brain- some regions have signal drop out). These 293 regions are listed in the file COBRE_fmri_regions.csv.
Also note that the values from the connectivity matrices are provided in vector format. To transform back to a connectivity matrix, you need to reshape the vectors.
E.g. in Matlab, use the command: matrix = reshape(connectivity_data(ind,:),[293,293])
where ind is an integer between 1 and 148 denoting the subject you are interested in. To check you have reshaped the matrix correctly, try plotting it- it should be a symmetric matrix with 1s on the diagonal.

COBRE_fmri_connectivity.csv- as above but saved as a .csv file.

COBRE_fmri_regions.csv- list of regions with fMRI data available (see above).

In [3]:
demographics = pd.read_csv("./data/COBRE_demographics.csv", sep=r'\s*,\s*', engine="python")
fmri_connectivity = pd.read_csv("./data/COBRE_fmri_connectivity.csv", sep=r'\s*,\s*', engine="python")
cortical_thickness = pd.read_csv("./data/COBRE_cortical_thickness.csv", sep=r'\s*,\s*', engine="python")
fmri_regions = pd.read_csv("./data/COBRE_fmri_regions.csv", sep=r'\s*,\s*', engine="python")
fmri_connectivity_np = genfromtxt("./data/COBRE_fmri_connectivity.csv", delimiter=',')

In [4]:
available_regions = np.concatenate((np.array([0]), fmri_regions['Region_no'].to_numpy() + 1))

# Quick data investigation

In [5]:
ages = demographics["age"]

c_pos_m = [0, 0, 0, 0]
c_con_m = [0, 0, 0, 0]

c_pos_f = [0, 0, 0, 0]
c_con_f = [0, 0, 0, 0]

for i in range(len(demographics["age"])):
    
    t_pos_m = 0
    t_neg_m = 0
    
    t_pos_f = 0
    t_neg_f = 0
    
    if(demographics["labels"][i] == -1):
        if(demographics["sex"][i] == -1):
            t_pos_m = 1
        else:
            t_pos_f = 1
    else:
        if(demographics["sex"][i] == -1):
            t_neg_m = 1
        else:
            t_neg_f = 1
            
    for j in range(4):
        min_age = (j+2)*10
        max_age = min_age + 10
        if(demographics["age"][i] > min_age and demographics["age"][i] < max_age):
            c_pos_m[j] += t_pos_m
            c_con_m[j] += t_neg_m
            c_pos_f[j] += t_pos_f
            c_con_f[j] += t_neg_f
         
            
print(c_pos_m)
print(c_con_m)

print(c_pos_f)
print(c_con_f)

[17, 13, 14, 10]
[20, 6, 6, 12]
[6, 6, 6, 3]
[1, 4, 1, 2]


In [6]:
subject_to_data = {} # Each item to a tuple (Age, True / False, fMRI_connectivity_matrix, cortical_thickness)
for p in range(len(demographics["SubjectID"])):
    patient = demographics.iloc[p]
    p_fmri = fmri_connectivity_np[p, :]
    if(patient["sex"] != -1):
        continue 
    subject_to_data[patient["SubjectID"]] = [patient["age"], (patient["labels"] == -1), np.reshape(p_fmri, (293, 293)), np.zeros(308)]
    
for p in range(len(cortical_thickness["Subject_ID"])):
    patient = cortical_thickness.iloc[p]
    if(patient["Subject_ID"] not in subject_to_data):
        continue
    
    subject_to_data[patient["Subject_ID"]][3] = patient.to_numpy()[2:]



In [7]:
X = []
y = []
for pid, l in subject_to_data.items():
    x_temp = cv2.resize(l[2]*255, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
    X.append(x_temp)
    y.append(int(l[1]))

X = np.array(X).astype(np.double)
y = np.array(y).astype(np.double)
    
train_x, val_x, train_y, val_y = train_test_split(X, y, test_size = 0.1)
print(train_x.shape, train_y.shape)
print(val_x.shape, val_y.shape)

train_x = train_x.reshape(102,1, 256, 256)
train_x  = torch.from_numpy(train_x)
val_x = val_x.reshape(12, 1, 256, 256)
val_x  = torch.from_numpy(val_x)

val_y = val_y.astype(int)
val_y = torch.from_numpy(val_y)
train_y = train_y.astype(int)
train_y = torch.from_numpy(train_y)



(102, 256, 256) (102,)
(12, 256, 256) (12,)


In [9]:
d = {}
for pid, l in subject_to_data.items():
    x_temp = cv2.resize(l[2]*255, dsize=(256, 256), interpolation=cv2.INTER_CUBIC)
    cat = (int(l[0] / 10)) - 2
    if cat not in [0, 1, 2, 3]:
        continue
    
    if cat not in d:
        d[cat] = []
    
    d[cat].append([int(l[1]), x_temp])
 
e = {}
for cat in d:
    l = np.array(d[cat])
    
    train_x2 = np.zeros(((math.ceil(.9* l[:, 1].shape[0])), l[:, 1][0].shape[0], l[:, 1][0].shape[1]))
    for i, item in enumerate(l[:, 1]):
        if(i >= train_x2.shape[0]):
            break
        train_x2[i, :, :] = item
    
    train_y2 = np.array(l[:train_x2.shape[0], 0], dtype=np.int16)
    train_x2 = np.expand_dims(train_x2, axis=1)
    print(train_x2.shape) 
    print(train_y2.shape)
    
    test_x2 = np.zeros((math.ceil(.1* l[:, 1].shape[0]), l[:, 1][0].shape[0], l[:, 1][0].shape[1]))
    c = 0
    for i, item in reversed(list(enumerate(l[:, 1]))):
        if(c >= test_x2.shape[0]):
            break
        test_x2[c, :, :] = item
        c+=1
    test_y2 = np.array(l[-1* test_x2.shape[0]:, 0], dtype=np.int16)
    test_x2 = np.expand_dims(test_x2, axis=1)
    print(test_x2.shape)
    print(test_y2.shape)
    print()
    e[cat] = (train_x2, train_y2, test_x2, test_y2)



(19, 1, 256, 256)
(19,)
(3, 1, 256, 256)
(3,)

(24, 1, 256, 256)
(24,)
(3, 1, 256, 256)
(3,)

(36, 1, 256, 256)
(36,)
(4, 1, 256, 256)
(4,)

(19, 1, 256, 256)
(19,)
(3, 1, 256, 256)
(3,)



<ipython-input-9-c28c1580e505>:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  l = np.array(d[cat])


## Classification

In [10]:
# Without doing k-means classification

def get_results(subject_to_data, min_age, max_age):
    X = []
    y = []
    kf = KFold(n_splits=10, shuffle=False)
    for pid, l in subject_to_data.items():
        if(l[0] >= min_age and l[0] < max_age):
            X.append(l[2].flatten())
            #X.append(l[3])
            y.append(int(l[1]))
    
    X = np.array(X)
    y = np.array(y)
    kf.get_n_splits(X)
    
    acc = []
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        svclassifier = SVC(kernel='linear')
        svclassifier.fit(X_train, y_train)
        y_pred = svclassifier.predict(X_test)
        acc.append(accuracy_score(y_test, y_pred))
    
    acc = np.array(acc)
    return (np.mean(acc), np.std(acc))
    #print(confusion_matrix(y_test,y_pred))
    #print(classification_report(y_test,y_pred))

print("All")    
print(get_results(subject_to_data, 0, 100))

print("20 to 30")    
print(get_results(subject_to_data, 20, 30))

print("30 to 40")    
print(get_results(subject_to_data, 30, 40))

print("40 to 50")    
print(get_results(subject_to_data, 40, 50))

print("50 to 60")    
print(get_results(subject_to_data, 50, 60))


All
(0.6068181818181818, 0.1554950771190666)
20 to 30
(0.6666666666666667, 0.15811388300841897)
30 to 40
(0.6666666666666666, 0.31622776601683794)
40 to 50
(0.4333333333333333, 0.3511884584284246)
50 to 60
(0.6333333333333333, 0.4068851871911235)


In [11]:
## K-means clustering
def k_means_get_results(subject_to_data, min_age, max_age):
    X = []
    y = []
    for pid, l in subject_to_data.items():
        if(l[0] >= min_age and l[0] < max_age):
            #X.append(l[2].flatten())
            X.append(l[3])
            y.append(int(l[1]))

    X = np.array(X)
    y = np.array(y)
    
    kmeans = GaussianMixture(n_components=5, covariance_type='diag').fit(X)
    y_kmeans = kmeans.predict(X)
    b2 = kmeans.predict_proba(X)
    
    kf = KFold(n_splits=10, shuffle=False)
    kf.get_n_splits(b2)
    
    acc = []
    for train_index, test_index in kf.split(b2):
        X_train, X_test = b2[train_index], b2[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        svclassifier = SVC(kernel='linear')
        svclassifier.fit(X_train, y_train)
        y_pred = svclassifier.predict(X_test)
        acc.append(accuracy_score(y_test, y_pred))
    
    acc = np.array(acc)
    return (np.mean(acc), np.std(acc))

In [12]:
print("All")    
print(k_means_get_results(subject_to_data, 0, 100))

print("20 to 30")    
print(k_means_get_results(subject_to_data, 20, 30))

print("30 to 40")    
print(k_means_get_results(subject_to_data, 30, 40))

print("40 to 50")    
print(k_means_get_results(subject_to_data, 40, 50))

print("50 to 60")    
print(k_means_get_results(subject_to_data, 50, 60))

All
(0.5856060606060606, 0.08886915816371432)
20 to 30
(0.7416666666666667, 0.16007810593582122)
30 to 40
(0.5333333333333333, 0.3559026084010437)
40 to 50
(0.55, 0.4153311931459037)
50 to 60
(0.35, 0.26299556396765833)


In [13]:
#t-SNE before k-means clustering
def tsne_k_means_get_results(subject_to_data, min_age, max_age):
    X = []
    y = []
    for pid, l in subject_to_data.items():
        if(l[0] >= min_age and l[0] < max_age):
            X.append(l[2].flatten())
            #X.append(l[3])
            y.append(int(l[1]))

    X = np.array(X)
    y = np.array(y)
    
    X = TSNE(n_components=2).fit_transform(X)
    
    kmeans = GaussianMixture(n_components=5, covariance_type='diag').fit(X)
    y_kmeans = kmeans.predict(X)
    b2 = kmeans.predict_proba(X)
    
    kf = KFold(n_splits=5, shuffle=True)
    kf.get_n_splits(b2)
    
    acc = []
    for train_index, test_index in kf.split(b2):
        X_train, X_test = b2[train_index], b2[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        svclassifier = SVC(kernel='linear')
        svclassifier.fit(X_train, y_train)
        y_pred = svclassifier.predict(X_test)
        acc.append(accuracy_score(y_test, y_pred))
    
    acc = np.array(acc)
    return (np.mean(acc), np.std(acc))

In [14]:
print("All")    
print(tsne_k_means_get_results(subject_to_data, 0, 100))

print("20 to 30")    
print(k_means_get_results(subject_to_data, 20, 30))

print("30 to 40")    
print(k_means_get_results(subject_to_data, 30, 40))

print("40 to 50")    
print(k_means_get_results(subject_to_data, 40, 50))

print("50 to 60")    
print(k_means_get_results(subject_to_data, 50, 60))

All
(0.6573122529644269, 0.09102587446466129)
20 to 30
(0.6666666666666667, 0.22360679774997894)
30 to 40
(0.4, 0.43588989435406733)
40 to 50
(0.41666666666666663, 0.30956959368344517)
50 to 60
(0.3833333333333333, 0.27938424357067015)


## Convolutional Neural Networks


In [15]:
class ConvModel(Module):
    def __init__(self):
        super(ConvModel, self).__init__()
        self.a1 = Conv2d(1, 4, kernel_size=3, stride=1, padding=1)
        self.a2 =BatchNorm2d(4)
        self.a3 = ReLU(inplace=True)
        self.a4 = MaxPool2d(kernel_size=2, stride=2)
        self.a5 = Conv2d(4, 4, kernel_size=3, stride=1, padding=1)
        self.a6 = BatchNorm2d(4)
        self.a7 = ReLU(inplace=True)
        self.a8 = MaxPool2d(kernel_size=2, stride=2)

        self.linear_layers = Sequential(
            Linear(4 * 64 * 64, 2)
        )

    def forward(self, x):
        x = self.a1(x)
        x = self.a2(x)
        x = self.a3(x)
        x = self.a4(x)
        x = self.a5(x)
        x = self.a6(x)
        x = self.a7(x)
        x = self.a8(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [16]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x.double()), Variable(train_y.long())
    #x_train = torch.from_numpy(x_train)
    #y_train = torch.from_numpy(y_train)
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)
    # converting the data into GPU format
    
    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train.double())
    output_val = model(x_val.double())

    # computing the training and validation loss
    loss_train = criterion(output_train, y_train.long())
    loss_val = criterion(output_val, y_val.long())
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [ ]:
model = ConvModel()
optimizer = Adam(model.parameters(), lr=0.07)
criterion = CrossEntropyLoss()
model = model.double()  
print(model)
# defining the number of epochs
n_epochs = 300
# empty list to store training losses
train_losses = []
# empty list to store validation losses
val_losses = []
# training the model
for epoch in range(n_epochs):
    train(epoch)
    
# plotting the training and validation loss
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.show()

with torch.no_grad():
    output = model(train_x.cpu().double())
    
softmax = torch.exp(output.cpu())
prob = softmax.numpy()
predictions = np.argmax(prob, axis=1)

# accuracy on training set
print(accuracy_score(train_y, predictions))

#with torch.no_grad():
#    output = model(val_x.cpu().double())

#softmax = torch.exp(output).cpu()
#prob = softmax.numpy()
#predictions = np.argmax(prob, axis=1)

# accuracy on validation set
#print(accuracy_score(val_y, predictions))

ConvModel(
  (a1): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (a2): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (a3): ReLU(inplace=True)
  (a4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (a5): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (a6): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (a7): ReLU(inplace=True)
  (a8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear_layers): Sequential(
    (0): Linear(in_features=16384, out_features=2, bias=True)
  )
)
Epoch :  1 	 loss : tensor(0.8709, dtype=torch.float64, grad_fn=<NllLossBackward>)
Epoch :  3 	 loss : tensor(420.6550, dtype=torch.float64, grad_fn=<NllLossBackward>)
Epoch :  5 	 loss : tensor(29.0627, dtype=torch.float64, grad_fn=<NllLossBackward>)
Epoch :  7 	 loss : tensor(34.8380, dtype=torch.float64, grad_fn=<NllLossBackward>)
Epoch :  9 	 los

# Model Agnostic Meta-Learning

In [ ]:
class MetaConvModel(MetaModule):
    def __init__(self):
        super(MetaConvModel, self).__init__()
        self.a1 = MetaConv2d(1, 4, kernel_size=3, stride=1, padding=1)
        self.a2 = MetaBatchNorm2d(4)
        self.a3 = ReLU(inplace=True)
        self.a4 = MaxPool2d(kernel_size=2, stride=2)
        self.a5 = MetaConv2d(4, 4, kernel_size=3, stride=1, padding=1)
        self.a6 = MetaBatchNorm2d(4)
        self.a7 = ReLU(inplace=True)
        self.a8 = MaxPool2d(kernel_size=2, stride=2)

        self.linear_layers = MetaSequential(
            MetaLinear(4 * 64 * 64, 2)
        )

    def forward(self, x):
        x = self.a1(x)
        x = self.a2(x)
        x = self.a3(x)
        x = self.a4(x)
        x = self.a5(x)
        x = self.a6(x)
        x = self.a7(x)
        x = self.a8(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [ ]:
model = MetaConvModel()
optimizer = Adam(model.parameters(), lr=0.07)
criterion = CrossEntropyLoss()

metalearner = ModelAgnosticMetaLearning(model,
                            optimizer,
                            first_order=False,
                            num_adaptation_steps=1,
                            step_size=.1,
                            loss_function=criterion,
                            device="cpu")

r = []
inner_losses = []
outer_losses = []
mean_outer_losses = []
accuracies_before = []
accuracies_after = []

for epoch in range(300):
    results = metalearner.train(e)
    #print(results)
    r.append(results)
    inner_losses.append(results["inner_losses"])
    outer_losses.append(results["outer_losses"])
    mean_outer_losses.append(results["mean_outer_loss"])
    accuracies_before.append(results["accuracies_before"])
    accuracies_after.append(results["accuracies_after"])
#results = metalearner.evaluate(val_x)
    


In [ ]:
#plt.plot(mean_outer_losses)
aaf = np.array(accuracies_after)
#print(aaf)
means = np.mean(aaf, axis=1)
#print(means)
m = np.convolve(means, np.ones(2)/2, mode='valid')
plt.plot(range(len(means)), means)
plt.ylim(0, 1)
plt.show()

In [ ]:
#plt.plot(mean_outer_losses)
#ols = np.array(outer_losses)
#print(aaf)
ols = np.mean(outer_losses, axis=1)
#print(means)
plt.plot(range(len(ols)), ols)

plt.show()